This compares model size and number of predictions for different model creation options and datasets.
I should really merge this into the dataset evaluation, but that will require changing the JSON export to include unconfirmed trips, and I just want to get this done now.

It assumes that the datasets are loaded into separate docker containers with ports exposed at 27071, 27018 and 27019.
It requires re-introduction of a **hack** 
https://github.com/e-mission/e-mission-server/commit/4cf770e3e30af617b89f9fe6007298afc37c0dd6
to allow re-loading of database connections


TODO: Clean this up later after cleaning up the export code

In [ ]:
import pandas as pd
import numpy as np
import logging

In [ ]:
import emission.core.get_database as edb

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq

In [ ]:
import emission.analysis.modelling.tour_model.data_preprocessing as old_preprocess
import emission.analysis.modelling.tour_model_first_only.data_preprocessing as new_preprocess

import emission.analysis.modelling.tour_model.get_users as old_users
import emission.analysis.modelling.tour_model_first_only.get_users as new_users

import emission.analysis.modelling.tour_model.evaluation_pipeline as old_ep
import emission.analysis.modelling.tour_model_first_only.evaluation_pipeline as new_ep

import emission.analysis.modelling.tour_model.load_predict as old_lp
import emission.analysis.modelling.tour_model_first_only.load_predict as new_lp

In [ ]:
import emission.analysis.modelling.tour_model.label_processing as lp

In [ ]:
# Let's just return a random split since their lengths should be roughly equal
# and that way we don't have to run the evaluation pipeline first
# and spend hours on it
# Copied version to allow us to override old implementation on the fly :)
def find_best_split_and_parameters(user,test_data):
    import random
    # find a random score
    best_split_idx = random.choice(range(len(test_data)))
    # use the position of best_score to find best_split
    best_split = test_data[best_split_idx]
    # use best_split_idx to find the best parameters
    low = best_split_idx * 100
    dist_pct = best_split_idx / 100
    return best_split,best_split_idx,low,dist_pct

In [ ]:
def create_location_map(trip_list, bins):
    bin_loc_feat = {}
    user_input_map = {}
    for i, curr_bin in enumerate(bins):
        # print(f"Considering {curr_bin} for trip list of length {len(trip_list)}")
        bin_trips = [trip_list[j] for j in curr_bin]
        # print(f"Considering {bin_trips} for bin {curr_bin}")
        x = old_preprocess.extract_features(bin_trips)
        bin_loc_feat[str(i)] = [feat[0:4] for feat in x]
    return bin_loc_feat

In [ ]:
def create_user_input_map(trip_list, bins):
    # map from bin index to user input probabilities
    # e.g. {"0": [{'labels': {'mode_confirm': 'drove_alone', 'purpose_confirm': 'work', 'replaced_mode': 'drove_alone'}, 'p': 1.0}]}
    user_input_map = {}
    for b, curr_bin in enumerate(bins):
        bin_trips = [trip_list[j] for j in curr_bin]
        user_label_df = pd.DataFrame([trip['data']['user_input'] for trip in bin_trips])
        # user_label_df = lp.map_labels(user_label_df)
        # compute the sum of trips in this cluster
        sum_trips = len(user_label_df)
        # compute unique label sets and their probabilities in one cluster
        # 'p' refers to probability
        unique_labels = user_label_df.groupby(user_label_df.columns.tolist()).size().reset_index(name='uniqcount')
        unique_labels['p'] = unique_labels.uniqcount / sum_trips
        labels_columns = user_label_df.columns.to_list()
        bin_label_combo_list = []
        for i in range(len(unique_labels)):
            one_set_labels = {}
            # e.g. labels_only={'mode_confirm': 'pilot_ebike', 'purpose_confirm': 'work', 'replaced_mode': 'walk'}
            labels_only = {column: unique_labels.iloc[i][column] for column in labels_columns}
            one_set_labels["labels"] = labels_only
            one_set_labels['p'] = unique_labels.iloc[i]['p']
            # e.g. one_set_labels = {'labels': {'mode_confirm': 'walk', 'replaced_mode': 'walk', 'purpose_confirm': 'exercise'}, 'p': 1.0}
            bin_label_combo_list.append(one_set_labels)
        user_input_map[str(b)] = bin_label_combo_list
    return user_input_map

In [ ]:
def find_bin(trip, bin_locations, radius):
    trip_feat = new_preprocess.extract_features([trip])[0]
    trip_loc_feat = trip_feat[0:4]
    first_round_label_set = list(bin_locations.keys())
    sel_fl = None
    for fl in first_round_label_set:
        # extract location features of selected bin
        sel_loc_feat = bin_locations[fl]
        # Check if start/end locations of the new trip and every start/end locations in this bin are within the range of
        # radius. If so, the new trip falls in this bin. Then predict the second round label of the new trip
        # using this bin's model
        if old_lp.in_bin(sel_loc_feat, trip_loc_feat, radius):
            sel_fl = fl
            break
    if not sel_fl:
        logging.debug(f"sel_fl = {sel_fl}, early return")
        return -1
    return sel_fl

In [ ]:
def get_dataset_stats(program, uuid_list):
    curr_dataset_validity_stats = []
    for u in uuid_list:
        curr_validity_stats = {"program": program, "user_id": u}
        trips = old_preprocess.read_data(u)
        # print([t for t in trips if "user_input" not in t["data"]])
        unlabeled_trips = [t for t in trips if t["data"]["user_input"] == {}]
        print(f"For {u} in {program}, read {len(trips)}")
        curr_validity_stats["all_trip_count_old"] = len(trips)
        curr_validity_stats["unlabeled_trip_count_old"] = len(unlabeled_trips)
        filter_trips = old_preprocess.filter_data(trips, 100)
        curr_validity_stats["valid_trip_count_old"] = len(filter_trips)
        curr_validity_stats["valid_user_old"] = old_users.valid_user(filter_trips, trips)
        if old_users.valid_user(filter_trips, trips):
            tune_idx, test_idx = old_preprocess.split_data(filter_trips)
            test_data = old_preprocess.get_subdata(filter_trips, tune_idx)
            best_split, best_split_idx, low, dist_pct = find_best_split_and_parameters(u,test_data)
            sim, bins, bin_trips, filter_trips = old_ep.first_round(best_split, 100)
            curr_validity_stats["bins_old"] = len(bins)
            curr_validity_stats["bin_trips_old"] = len(bin_trips)
            curr_validity_stats["model_input_trips_old"] = len(bin_trips)
            print(f"Creating model maps for {bins} and {len(bin_trips)}")
            bin_loc_feat = create_location_map(filter_trips, bins)
            user_input_map = create_user_input_map(filter_trips, bins)
            pred_bin_list = [find_bin(t, bin_loc_feat, 100) for t in trips]
            print(f"Found old predicted bins {pred_bin_list}")
            curr_validity_stats["predicted_trips_all_old"] = np.count_nonzero(pd.Series(pred_bin_list) != -1)
            pred_bin_list = [find_bin(t, bin_loc_feat, 100) for t in unlabeled_trips]
            print(f"Found old unlabeled predicted bins {pred_bin_list}")
            curr_validity_stats["predicted_trips_unlabeled_old"] = np.count_nonzero(pd.Series(pred_bin_list) != -1)
            user_input_pred_list = [user_input_map[pb] for pb in pred_bin_list if pb != -1]
            # print(f"User predictions for unlabeled trips are {user_input_pred_list}")
        else:
            curr_validity_stats["bins_old"] = 0
            curr_validity_stats["bin_trips_old"] = 0
            curr_validity_stats["model_input_trips_old"] = 0

        # print(f"Finished iterating over old, starting with new....")
        trips = new_preprocess.read_data(u)
        unlabeled_trips = [t for t in trips if t["data"]["user_input"] == {}]

        curr_validity_stats["all_trip_count_new"] = len(trips)
        curr_validity_stats["unlabeled_trip_count_new"] = len(unlabeled_trips)
        filter_trips = new_preprocess.filter_data(trips, 500)
        curr_validity_stats["valid_trip_count_new"] = len(filter_trips)
        curr_validity_stats["valid_user_new"] = new_users.valid_user(filter_trips, trips)
        sim, bins, bin_trips, filter_trips = new_ep.first_round(filter_trips, 500)
        curr_validity_stats["bins_new"] = len(bins)
        curr_validity_stats["bin_trips_new"] = len(bin_trips)
        curr_validity_stats["model_input_trips_new"] = len(bin_trips)
        print(f"Creating model maps for {len(bin_trips)}")
        bin_loc_feat = create_location_map(filter_trips, bins)
        user_input_map = create_user_input_map(filter_trips, bins)
        pred_bin_list = [find_bin(t, bin_loc_feat, 500) for t in trips]
        print(f"Found new predicted bins {pred_bin_list}")
        curr_validity_stats["predicted_trips_all_new"] = np.count_nonzero(pd.Series(pred_bin_list) != -1)
        pred_bin_list = [find_bin(t, bin_loc_feat, 500) for t in filter_trips]
        assert len(pred_bin_list) == len(filter_trips)
        pred_bin_list = [find_bin(t, bin_loc_feat, 500) for t in unlabeled_trips]
        print(f"Found new unlabeled predicted bins {pred_bin_list}")
        curr_validity_stats["predicted_trips_unlabeled_new"] = np.count_nonzero(pd.Series(pred_bin_list) != -1)

        curr_dataset_validity_stats.append(curr_validity_stats)
    return curr_dataset_validity_stats

In [ ]:
model_validity_stats = []

In [ ]:
model_validity_stats = model_validity_stats + get_dataset_stats("minipilot", esta.TimeSeries.get_uuid_list())

In [ ]:
esta.TimeSeries._reset_url("localhost:27018")

In [ ]:
model_validity_stats = model_validity_stats + get_dataset_stats("nrel-lh", esta.TimeSeries.get_uuid_list())

In [ ]:
edb.get_profile_db().distinct("client")

In [ ]:
esta.TimeSeries._reset_url("localhost:27019")

In [ ]:
model_validity_stats = model_validity_stats + get_dataset_stats("stage", esta.TimeSeries.get_uuid_list())

In [ ]:
model_validity_stats_df = pd.DataFrame(model_validity_stats); model_validity_stats_df

In [ ]:
model_validity_stats_df.plot(kind="bar", y=["all_trip_count_old", "all_trip_count_new"],figsize=(20,10))

In [ ]:
model_validity_stats_df.plot(kind="bar", y=["model_input_trips_old", "model_input_trips_new"],figsize=(20,10))

In [ ]:
model_validity_df = pd.DataFrame(model_validity_stats)
model_validity_df[["all_trip_count_old", "unlabeled_trip_count_old", "model_input_trips_old", "model_input_trips_new", "predicted_trips_all_old", "predicted_trips_unlabeled_old", "predicted_trips_all_new", "predicted_trips_unlabeled_new"]]

In [ ]:
model_validity_df.plot.bar(y=["all_trip_count_old", "predicted_trips_all_old", "predicted_trips_all_new"], figsize=(20,5))

In [ ]:
model_validity_df.plot.bar(y=["unlabeled_trip_count_old", "predicted_trips_unlabeled_old", "predicted_trips_unlabeled_new"], figsize=(20,5))

In [ ]:
model_validity_df["unlabeled_predict_pct_new"] = model_validity_df.predicted_trips_unlabeled_new/model_validity_df.unlabeled_trip_count_new

In [ ]:
model_validity_df["unlabeled_predict_pct_old"] = model_validity_df.predicted_trips_unlabeled_old/model_validity_df.unlabeled_trip_count_old

In [ ]:
model_validity_df.plot(kind="bar", y=["unlabeled_predict_pct_old", "unlabeled_predict_pct_new"], figsize=(20,5))

In [ ]:

fig = plt.Figure(figsize=(10,5))
axarr = fig.subplots(1,2)
model_validity_df.plot(kind="scatter", x="valid_trip_count_new", y="unlabeled_predict_pct_old", color="red", ax=axarr[0])
model_validity_df.plot(kind="scatter", x="valid_trip_count_new", y="unlabeled_predict_pct_new", color="green", ax=axarr[1])
fig

In [ ]:
model_validity_df.query("unlabeled_predict_pct_new < 0.2")[["program", "valid_trip_count_new", "unlabeled_predict_pct_old", "unlabeled_predict_pct_new"]].to_markdown()

In [ ]:
model_validity_df.query("unlabeled_predict_pct_new < 0.01")[["program", "all_trip_count_new", "unlabeled_trip_count_new", "valid_trip_count_new", "unlabeled_predict_pct_old", "unlabeled_predict_pct_new"]]